# Knapsack Problems

In [1]:
from pyomo.environ import *
import numpy as np

## 0-1 Knapsack: Example

Given a set of $N=5$ items with their *weights* $w_i$ kg and *values* $v_i$ pesos as follows:

|         | Weight | Value |
| ------- | ------ | ----- |
| Item 1  | 4      | 8     |
| Item 2  | 3      | 5     |
| Item 3  | 6      | 9     |
| Item 4  | 2      | 6     |
| Item 5  | 1      | 7     |


Find a subset of the items with the maximum total value that can fit in a bag with a maximum capacity of $W_{max}=8$.

In [2]:
# Create the model object

def create_01knapsack_mdl(N, W, V, Wmax):
    # N    = set of items
    # W    = vector of weights
    # V    = vector of values
    # Wmax = maximum capacity
    
    model = ConcreteModel(name="knapsack")

    # Declare decision variables and parameters
    model.x = Var(N, domain=Binary)
    model.w = Param(N, initialize=lambda mdl, i: W[i-1])
    model.v = Param(N, initialize=lambda mdl, i: V[i-1])
    
    def obj_rule(mdl):
        return sum(mdl.x[i]*mdl.v[i] for i in N)
    
    def capacity_rule(mdl):
        return sum(mdl.x[i]*mdl.w[i] for i in N) <= Wmax
    
    model.obj = Objective(rule=obj_rule, sense=maximize)
    model.cap = Constraint(rule=capacity_rule)
    
    return model

# Specify the data
N = RangeSet(5)
W = [4, 3, 6, 2, 1]
V = [8, 5, 9, 6, 7]
Wmax = 8

# Solve the model
model = create_01knapsack_mdl(N, W, V, Wmax)
solver = SolverFactory("glpk") 
res = solver.solve(model)
model.display() 
res.write()

Model knapsack

  Variables:
    x : Size=5, Index=[1:5]
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  21.0

  Constraints:
    cap : Size=1
        Key  : Lower : Body : Upper
        None :  None :  7.0 :   8.0
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown


## Fractional Knapsack: Example

Given a set of $N=6$ items with their *weights* $w_i$ kg and *values* $v_i$ pesos as follows:

|         | Weight | Value |
| ------- | ------ | ----- |
| Item 1  | 5      | 8     |
| Item 2  | 7      | 3     |
| Item 3  | 4      | 6     |
| Item 4  | 3      | 11    |
| Item 5  | 2      | 4     |
| Item 6  | 4      | 5     |

Only one of each item is available. Find the fraction of each item whose total value is maximum but can also fit in a bag with a maximum capacity of $W_{max}=4.25$. 

In [3]:
# Create the model object

def create_fracknapsack_mdl(N, W, V, Wmax):
    # N    = set of items
    # W    = vector of weights
    # V    = vector of values
    # Wmax = maximum capacity
    
    model = ConcreteModel(name="knapsack")

    # Declare decision variables and parameters
    model.x = Var(N, domain=NonNegativeReals, bounds=(0, 1))
    model.w = Param(N, initialize=lambda mdl, i: W[i-1])
    model.v = Param(N, initialize=lambda mdl, i: V[i-1])
    
    def obj_rule(mdl):
        return sum(mdl.x[i]*mdl.v[i] for i in N)
    
    def capacity_rule(mdl):
        return sum(mdl.x[i]*mdl.w[i] for i in N) <= Wmax
    
    model.obj = Objective(rule=obj_rule, sense=maximize)
    model.cap = Constraint(rule=capacity_rule)
    
    return model

# Specify the data
N = RangeSet(6)
W = [5, 7, 4, 3, 2, 4]
V = [8, 3, 6, 11, 4, 5]
Wmax = 4.25

# Solve the model
model = create_fracknapsack_mdl(N, W, V, Wmax)
solver = SolverFactory("glpk") 
res = solver.solve(model)
model.display()
res.write()

Model knapsack

  Variables:
    x : Size=6, Index=[1:6]
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : NonNegativeReals
          2 :     0 :   0.0 :     1 : False : False : NonNegativeReals
          3 :     0 :   0.0 :     1 : False : False : NonNegativeReals
          4 :     0 :   1.0 :     1 : False : False : NonNegativeReals
          5 :     0 : 0.625 :     1 : False : False : NonNegativeReals
          6 :     0 :   0.0 :     1 : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  13.5

  Constraints:
    cap : Size=1
        Key  : Lower : Body : Upper
        None :  None : 4.25 :  4.25
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# -------------------------------------------------

## Bounded Knapsack: Example

Given a set of $N=6$ distinct items with their *weights* $w_i$ kg, *values* $v_i$ pesos, and number of available copies $c_i$ as follows:

|         | Weight | Value | Copies |
| ------- | ------ | ----- | ------ |
| Item 1  | 5      | 8     | 2      |
| Item 2  | 7      | 3     | 2      |
| Item 3  | 4      | 6     | 1      |
| Item 4  | 3      | 11    | 1      |
| Item 5  | 2      | 4     | 2      |
| Item 6  | 4      | 5     | 4      |

Find the number of copies of each item with maximum total value but can also fit in a bag with a maximum capacity of $W_{max}=20$. 

In [4]:
# Create the model object

def create_bndknapsack_mdl(N, W, V, C, Wmax):
    # N    = set of items
    # W    = vector of weights
    # V    = vector of values
    # C    = max no. of copies of each item
    # Wmax = maximum capacity
    
    model = ConcreteModel(name="knapsack")

    # Declare decision variables and parameters
    model.x = Var(N, domain=Integers, bounds=lambda mdl, i: (0, C[i-1]))
    model.w = Param(N, initialize=lambda mdl, i: W[i-1])
    model.v = Param(N, initialize=lambda mdl, i: V[i-1])
    
    def obj_rule(mdl):
        return sum(mdl.x[i]*mdl.v[i] for i in N)
    
    def capacity_rule(mdl):
        return sum(mdl.x[i]*mdl.w[i] for i in N) <= Wmax
    
    model.obj = Objective(rule=obj_rule, sense=maximize)
    model.cap = Constraint(rule=capacity_rule)
    
    return model

# Specify the data
N = RangeSet(6)
W = [5, 7, 4, 3, 2, 4]
V = [8, 3, 6, 11, 4, 5]
C = [2, 2, 1, 1, 2, 4]
Wmax = 20

# Solve the model
model = create_bndknapsack_mdl(N, W, V, C, Wmax)
solver = SolverFactory("glpk") 
res = solver.solve(model)
model.display()
res.write()

Model knapsack

  Variables:
    x : Size=6, Index=[1:6]
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     2 : False : False : Integers
          2 :     0 :   0.0 :     2 : False : False : Integers
          3 :     0 :   1.0 :     1 : False : False : Integers
          4 :     0 :   1.0 :     1 : False : False : Integers
          5 :     0 :   2.0 :     2 : False : False : Integers
          6 :     0 :   1.0 :     4 : False : False : Integers

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  38.0

  Constraints:
    cap : Size=1
        Key  : Lower : Body : Upper
        None :  None : 20.0 :  20.0
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ------------